In [1]:
!pip install --quiet -U langgraph langchain_ollama langchain_core langchain_openai mermaid-python
import os
os.environ["OPENAI_API_KEY"] = os.environ.get("OPEN_AI_KEY")

# Goals
- We can extend a router, into a generic agent architecture. We return a `ToolMessage` to a user, when we call a tool.
- What happens if we pass a `ToolMessage` back to a model?

# ReAct
- **ReAct** is a general agent architecture:
    - `act` - let the model call specific tools
    - `observe` - pass the tool output back to the model
    - `reason` - let the model reason about the tool output to decide what to do next.

In [2]:
from langchain_openai import ChatOpenAI

def multiply(a: int, b: int) -> int:
    """
    Multiply a and b
    :param a: integer
    :param b: integer
    :return: integer product
    """
    return a * b

def add(a: int, b: int) -> int:
    """
    Adds a and b
    :param a: integer
    :param b: integer
    :return: integer addition
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add, multiply, divide]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [3]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}